In [1]:
#load packages
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#OBS need to be made usable
'''class BlameDetectorDa(object):

    def __init__(self, model_path, max_length, batch_size = None):

        self.model_path = model_path
        self.max_length = max_length
        self.batch_size = batch_size

        self.model_initialization()

        return

    def model_initialization(self):
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)

        self.model.eval()
            
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)

        print(f"Model loaded successfully on {self.device}")

        return

    def predict(self):
        """Make a prediction on a single text input."""
        # Tokenize input
        inputs = self.tokenizer(
            self.text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Make prediction
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()
        
        return predicted_class, confidence, probabilities[0].cpu().numpy()

    def run_prediction(self, text):

        self.text = text
        predicted_class, confidence, probs = self.predict()
            
        return predicted_class, confidence
'''


In [ ]:
'''example = "Goddad min kære ven, jeg kan virkelig godt lide dig."

BD = BlameDetectorDa()
BD.run_prediction(text = example)'''

In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from typing import Union, List, Tuple

class BlameDetectorDa(object):
    """
    Danish Blame Detection classifier with support for single and batch predictions.
    """

    def __init__(self, model_path: str, max_length: int = 512, batch_size: int = None):
        """
        Initialize the BlameDetectorDa classifier.
        
        Args:
            model_path: Path to the pretrained model
            max_length: Maximum sequence length for tokenization
            batch_size: Batch size for batch predictions (None for single predictions)
        """
        self.model_path = model_path
        self.max_length = max_length
        self.batch_size = batch_size

        self.model_initialization() #change when model is ready

    def model_initialization(self):
        """Initialize the model, tokenizer, and device."""
        print(f"Loading model from {self.model_path}...")
        
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_path,
            trust_remote_code=True  # Add this for ModernBERT
        )
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)

        self.model.eval()
            
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)  # Fixed: was 'device', should be 'self.device'

        print(f"Model loaded successfully on {self.device}")

    def model_initialization_prelim(self):
        print(f"Loading model from {self.model_path}...")
    
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_name)
        
        # Load base model with classification head
        print(f"Loading base model: {self.base_model_name}")
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.base_model_name,
            num_labels=self.num_labels,
            trust_remote_code=True
        )
        
        # Load LoRA adapters
        print("Loading LoRA adapters...")
        self.model = PeftModel.from_pretrained(self.model, self.model_path)
        
        # Merge LoRA weights with base model for faster inference (optional)
        print("Merging LoRA weights...")
        self.model = self.model.merge_and_unload()
        
        # Set model to evaluation mode
        self.model.eval()
        
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        
        print(f"Model loaded successfully on {self.device}")
        return
        

    def predict_single(self, text: str) -> Tuple[int, float, np.ndarray]:
        """
        Make a prediction on a single text input.
        
        Args:
            text: Input text to classify
            
        Returns:
            Tuple of (predicted_class, confidence, probabilities)
        """
        # Tokenize input
        inputs = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Make prediction
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()
        
        return predicted_class, confidence, probabilities[0].cpu().numpy()

    def predict_batch(self, texts: List[str]) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Make predictions on a batch of text inputs.
        
        Args:
            texts: List of input texts to classify
            
        Returns:
            Tuple of (predicted_classes, confidences, all_probabilities)
        """
        # Tokenize all inputs
        inputs = self.tokenizer(
            texts,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Make predictions
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_classes = torch.argmax(probabilities, dim=1).cpu().numpy()
            confidences = probabilities.max(dim=1).values.cpu().numpy()
            all_probs = probabilities.cpu().numpy()
        
        return predicted_classes, confidences, all_probs

    def run_prediction(
        self, 
        text: Union[str, List[str]]
    ) -> Union[Tuple[int, float], Tuple[np.ndarray, np.ndarray]]:
        """
        Run prediction on single text or batch of texts.
        
        Args:
            text: Single text string or list of text strings
            
        Returns:
            For single text: (predicted_class, confidence)
            For batch: (predicted_classes, confidences)
        """
        # Check if input is a list (batch) or single text
        if isinstance(text, list):
            # Batch prediction
            if self.batch_size is not None and len(text) > self.batch_size:
                # Process in batches
                all_classes = []
                all_confidences = []
                
                for i in range(0, len(text), self.batch_size):
                    batch = text[i:i + self.batch_size]
                    classes, confidences, _ = self.predict_batch(batch)
                    all_classes.extend(classes)
                    all_confidences.extend(confidences)
                
                return np.array(all_classes), np.array(all_confidences)
            else:
                # Single batch prediction
                predicted_classes, confidences, _ = self.predict_batch(text)
                return predicted_classes, confidences
        else:
            # Single text prediction
            predicted_class, confidence, _ = self.predict_single(text)
            return predicted_class, confidence


'''# Example usage

# Initialize detector
MODEL_PATH = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_merged"
detector = BlameDetectorDa(
    model_path=MODEL_PATH,
    max_length=512,
    batch_size=8  # Process 8 texts at a time
)

print("Single Prediction Test")


# Single prediction
single_text = "Dette er en test sætning."
pred_class, confidence = detector.run_prediction(single_text)
print(f"\nText: {single_text}")
print(f"Predicted Class: {pred_class}")
print(f"Confidence: {confidence:.4f}")

print("Batch Prediction Test")

# Batch prediction
batch_texts = [
    "Første test sætning.",
    "Anden test sætning.",
    "Tredje test sætning.",
    "Fjerde test sætning.",
]

pred_classes, confidences = detector.run_prediction(batch_texts)

print("\nBatch Results:")
for i, (text, cls, conf) in enumerate(zip(batch_texts, pred_classes, confidences), 1):
    print(f"\n{i}. Text: {text}")
    print(f"   Class: {cls}, Confidence: {conf:.4f}")'''

'# Example usage\n\n# Initialize detector\nMODEL_PATH = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_merged"\ndetector = BlameDetectorDa(\n    model_path=MODEL_PATH,\n    max_length=512,\n    batch_size=8  # Process 8 texts at a time\n)\n\nprint("Single Prediction Test")\n\n\n# Single prediction\nsingle_text = "Dette er en test sætning."\npred_class, confidence = detector.run_prediction(single_text)\nprint(f"\nText: {single_text}")\nprint(f"Predicted Class: {pred_class}")\nprint(f"Confidence: {confidence:.4f}")\n\nprint("Batch Prediction Test")\n\n# Batch prediction\nbatch_texts = [\n    "Første test sætning.",\n    "Anden test sætning.",\n    "Tredje test sætning.",\n    "Fjerde test sætning.",\n]\n\npred_classes, confidences = detector.run_prediction(batch_texts)\n\nprint("\nBatch Results:")\nfor i, (text, cls, conf) in enumerate(zip(batch_texts, pred_classes, confidences), 1):\n    print(f"\n{i}. Text: {text}")\n    print(f"   Class: {cls}, Confide

In [3]:
import json
from tqdm import tqdm  # for progress bar (optional)

MODEL_PATH = '/work/RuneEgeskovTrust#9638/Bachelor/best_model_epoch2'#runetrust/mmBlameBERT-pol-DA'#"/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_model"



detector = BlameDetectorDa(
    model_path=MODEL_PATH,
    max_length=1024,
    batch_size=2  # Process 2 texts at a time
)


Loading model from /work/RuneEgeskovTrust#9638/Bachelor/best_model_epoch2...


2025-10-28 09:43:03.715108: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-28 09:43:10.439933: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-28 09:43:50.691450: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Some weights of ModernBertForSequenceClassification were not initial

Model loaded successfully on cuda


In [32]:
# ---- 1. Load JSON dataset ----
import json
json_path = "/work/RuneEgeskovTrust#9638/Bachelor/training_data/subset_4_5_cleaned_training_data.json"#Bachelor_project/Model_data/validation_set.json"

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)


In [33]:


# ---- 2. Extract texts ----
texts = [item["text"] for item in data if "text" in item]

# ---- 3. Run predictions in batches ----
pred_classes, confidences = detector.run_prediction(texts)


# ---- 4. Attach predictions back to data ----
for item, cls, conf in zip(data, pred_classes, confidences):
    item["predicted_class"] = int(cls)
    item["confidence"] = float(conf)


#save data



In [15]:
correct = 0
for entry in data:
    if entry["label"] == entry["predicted_class"]:
        correct+=1


correct/len(data)

0.6591666666666667

In [47]:
tokenizer = AutoTokenizer.from_pretrained(lora_adapter_path)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# ----------------------------
# 1. Define model and adapter paths
# ----------------------------
base_model_name = "jhu-clsp/mmBERT-base"        # The base model (e.g. BERT)
lora_adapter_path = "/work/RuneEgeskovTrust#9638/Bachelor/best_model_epoch2"  # Path to LoRA adapter checkpoint
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------
# 2. Load tokenizer and model
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained(lora_adapter_path)
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name)

# ----------------------------
# 3. Load LoRA adapter into base model
# ----------------------------
model = PeftModel.from_pretrained(base_model, lora_adapter_path)
model.to(device)
model.eval()

# ----------------------------
# 4. Example sentences
# ----------------------------
sentences = texts 
'''
[
    "The food was absolutely wonderful.",
    "I really disliked the movie."
]
'''
# ----------------------------
# 5. Tokenize input
# ----------------------------
inputs = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
).to(device)

# ----------------------------
# 6. Run inference
# ----------------------------
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# ----------------------------
# 7. Print results
# ----------------------------
for sent, pred in zip(sentences, predictions):
    print(f"Sentence: {sent}")
    print(f"Predicted class: {pred.item()}")

# ---- 4. Attach predictions back to data ----
for sent, pred in zip(sentences, predictions):
    item["predicted_class"] = int(pred)
    #item["confidence"] = float(conf)


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentence: Vi er selvfølgelig enige i, at der skal være ret og  rimelighed i velfærden.
Predicted class: 1
Sentence: Længere nede i samme artikel siger hun, at CD  blev dannet som: »en protest mod den venstredrejning, der  skete i Socialdemokratiet, men i mellemtiden er  Socialdemokratiet jo blevet borgerliggjort, og nu er det os,  der skal trække Socialdemokratiet i den anden retning.
Predicted class: 1
Sentence: For selv om man  ganske vist gennemførte deportationen to uger før den danske  grundlovsændring trådte i kraft - så vidt jeg husker forløbet  - var det jo danske myndigheder, der var med til at foretage  den deportation, danske myndigheder, som var ansvarlige over  for dette Folketing, og også før grundloven af 1953 var  regeringen ansvarlig over for dette Folketing.  Så det  var  danske, demokratisk valgte  myndigheder, der havde ansvaret for de forhold og også for de  forhold, der var i Island og på Færøerne i perioden før Anden  Verdenskrig, og som jo førte til, at Island t

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

texts = [item["text"] for item in data if "text" in item]
true_labels = [item["label"] for item in data if "label" in item]

batch_size = 16
predictions = []

for i in range(0, len(text#s), batch_size):
    batch_texts = texts[i:i+batch_size]
    inputs = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# ----------------------------
# 5. Evaluate performance
# ----------------------------

flipped_preds = [1 - p for p in predictions]


acc = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average="macro")

print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(true_labels, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predictions))

Accuracy: 0.5108
F1 Score: 0.3579

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.02      0.04      1800
           1       0.51      1.00      0.67      1800

    accuracy                           0.51      3600
   macro avg       0.73      0.51      0.36      3600
weighted avg       0.73      0.51      0.36      3600


Confusion Matrix:
[[  41 1759]
 [   2 1798]]


In [52]:
print(model.config.id2label)
print(model.config.label2id)


{0: 'LABEL_0', 1: 'LABEL_1'}
{'LABEL_0': 0, 'LABEL_1': 1}


In [59]:


examples = ["Når man har siddet og hørt på hr. Lars Løkke Rasmussens  indlæg, så må man spørge: Hvor var Venstre henne, da  fædrelandets hovedstad havde brug for dem?", "Folket skal have mulighed for flere penge mellem hænderne."]
inputs = tokenizer(examples, return_tensors="pt", padding=True, truncation=True).to(device)
with torch.no_grad():
    logits = model(**inputs).logits
print(torch.softmax(logits, dim=-1))


tensor([[0.2061, 0.7939],
        [0.0529, 0.9471]], device='cuda:0')
